# Union Find木
グループ分けを効率的に管理できるデータ構造。  
グループごとに根付き木を構成し、その根同士を効率的に結合していくことで、グループを管理する。  
あるノードが同じグループかどうかを検証するには、そのノードの根が同じかどうかを判定する  

### C++での実装
```cpp
class UnionFind {
 public:
  explicit UnionFind(size_t n) : parents(n, -1) {}

  // 頂点 i のrootのインデックスを返す
  int find(int i) {
    if (parents[i] < 0) {
      return i;
    }
    parents[i] = find(parents[i]);
    return parents[i];
  }

  void unite(int a, int b) {
    a = find(a);
    b = find(b);
    if (a != b) {
      // union by size(小さい方を子にしてメモリを削減)
      if (parents[a] < parents[b]) swap(a, b);
      parents[a] += parents[b];
      parents[b] = a;
    }
  }

  bool is_same(int a, int b) { return (find(a) == find(b)); }

  int size(int i) { return -parents[find(i)]; }

 private:
  vector<int> parents;
};
```

### Pythonでの実装

In [2]:
class UnionFind:
    def __init__(self, n) -> None:
        self.n = n
        self.parent = [-1]*n
        self.rank = [1]*n

    # 親を探す
    def find(self, v):
        if self.parent[v] < 0:
            return v
        else:
            self.parent[v] = self.find(self.parent[v])
            return self.parent[v]

    def is_same(self, x, y):
        return self.find(x) == self.find(y)

    # 指定のノードを含むグループのサイズを返す
    def size(self, v):
        return -self.parent[self.find(v)]

    def unite(self, x, y):
        x_root = self.find(x)
        y_root = self.find(y)

        # 既に同じ周濠に属している場合
        if x_root == y_root:
            return False

        # 結合の際に、rank数が高い方を親にすることで、findメソットの計算量がO(logN)まで削減することができる
        if self.rank[x_root] > self.rank[y_root]:
            x_root, y_root = y_root, x_root
        if self.rank[x_root] == self.rank[y_root]:
            self.rank[y_root] += 1
        
        # uniteするグループの数を取り込むことでグループ数を記録する
        self.parent[y_root] += self.parent[x_root]
        self.parent[x_root] = y_root

def main():
    N, Q = map(int, stdin.readline().split())
    uf_tree = UnionFind(N)
    for _ in range(Q):
        t, u, v = map(lambda x: int(x)-1, stdin.readline().split())
        if t == 0:
            uf_tree.unite(u, v)
        elif t == 1:
            if uf_tree.find(u) == uf_tree.find(v):
                print("Yes")
            else:
                print("No")


### Union by size
根付き木同士を結合する際、**大きい方の木に、小さい方を取り組むように結合(union by size)**すると、  
木が大きなることが避けられ、O(logN)時間で、ノードの根を取得することができる

### parent配列を用いたグループのsize管理
以下のブログを参考に、グループのルートノードにマイナスでグループのサイズを保持するテクニックを用いた。  
https://yaakublog.com/union_find

## Union Findを使った有名アルゴリズム
### Kruskal algorithm(クラスカル法)
最小全域木(MST: Minimum spanning tree)を求めるためのアルゴリズム。全ての辺を、小さい順に並び替え、小さい方から、閉路を作らないようにUnion Findで管理しながら辺をつなげていくと、最小全域木が完成する。

In [1]:
def main():
    N, M = map(int, stdin.readline().split())
    graph = [[] for _ in range(N)]
    paths = []
    for _ in range(M):
        A, B, C = map(int, stdin.readline().split())
        graph[A-1].append([B-1, C])
        graph[B-1].append([A-1, C])
        paths.append([A-1, B-1, C])

    # Kruskal algorithm
    # 辺を小さい順に並び替える
    paths.sort(key=lambda x: x[2])
    uf_tree = UnionFind(N)
    ans = 0
    # 小さい辺から順に、閉路を作らない場合に、その辺がMSTを作る辺として採用される。
    for path in paths:
        A, B, C = path
        if uf_tree.is_same(A, B) == False:
            ans += C
            uf_tree.unite(A, B)

    print(ans)
